In [4]:
import requests
import re
from bs4 import BeautifulSoup as bs

# Метод получения до n ссылок c помощью regex
# из habr страницы, заданной через url
def get_posts(n, url):
    post_url_regex = '.+habr.com/ru/post/\d+/$'
    posts = set()
    page = requests.get(url)
    if page.status_code == 200:
        data = bs(page.content, features="html.parser")
        for link in data.findAll('a', href=True):
            href = link['href']
            if re.match(post_url_regex, href):
                posts.add(href)
            if len(posts) >= n:
                break
    return posts

In [11]:
BASE_URL = 'https://habr.com/ru/flows/develop/top/alltime/'
post_urls = set()
counter = 1
# проходим по habr страницам c топами и вытаскиваем ссылки на конкретные посты (до 100) 
while len(post_urls) < 100:
    post_urls = post_urls.union(get_posts(n=100 - len(post_urls), url=BASE_URL + f'page{counter}/'))
    counter += 1
print(f'posts_url length = {len(post_urls)}')

posts_url length = 100


In [15]:
from zipfile import ZipFile, ZIP_DEFLATED
# Сохраняем страницы с постами из ссылок в zip архив
counter = 1
with ZipFile('pages.zip', 'w', ZIP_DEFLATED) as zipFile:
    for url in post_urls:
        page = requests.get(url)
        zipFile.writestr(f'document_{counter}.html', page.text)
        counter += 1
print(f'saved files in pages.zip = {counter - 1}')

saved files in pages.zip = 100


In [14]:
# Сохраняем список ссылок c номерами в index.txt
counter = 1
with open('index.txt', "w", encoding="utf-8") as file:
    for url in post_urls:
        file.write(f'document_{counter}. {url}\n')
        counter += 1
print(f'saved links in index.txt = {counter - 1}')

saved links in index.txt = 100
